## Testing sampling time

In [1]:
import sys
import argparse
import h5py
import warnings
from functools import reduce
import operator
import multiprocessing
from copy import deepcopy
import pdb
import time

import numpy as np
import pandas as pd
import scipy.stats

from populations import bbh_models, gw_obs
from sample import sample
from plot import msplot


"""
l shows where you are in the code
n run next line
c continue code
"""

# --- Save certain boolean arguments
verbose=True
make_plots=False
use_flows=True
# see if we're using GW observations
gwobs = True
model0='gwobs'

channels = ['CE','CHE','GC','NSC','SMT']
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path='/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
flow_model_filename = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/'
params =['mchirp', 'q','chieff', 'z']
uncertainty = 'posteriors'
Nsamps=100
prior=None

# --- Useful functions for accessing items in dictionary
def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)
def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value

# --- Load in models/kdemodels into dict structure: models[model][channel]
# Construct both "underlying" KDE models (anything that has Pdet>0) and "detectable" KDE models
# `normalize` argument normalizes KDEs on unit cube
params = list(np.asarray(params).flatten())
if use_flows:
    model_names, pop_models = bbh_models.get_models(file_path, channels, params, use_flows=use_flows,detectable=False)
    #TO CHANGE - add detectable models?
    model_names.sort()
    hyperparams = sorted(list(set([x.split('/', 1)[1] for x in model_names])))
    Nhyper = np.max([len(x.split('/')) for x in hyperparams])


channels = sorted(list(pop_models.keys()))

# create dict for the hyperparameters at each level
hyperparam_dict  = {}
hyperidx=0
while hyperidx < Nhyper:
    hyperidx_with_Nhyper = np.argwhere(np.asarray([len(x.split('/')) for x in hyperparams])>hyperidx).flatten()
    hyperparams_at_level = sorted(set([x.split('/')[hyperidx] for x in np.asarray(hyperparams)[hyperidx_with_Nhyper]]))
    hyperparam_dict[hyperidx] = hyperparams_at_level
    hyperidx += 1

if verbose:
    print("")
    print("Formation channels: " + ", ".join(channels))
    print("Astrophysical models: " + ", ".join(model_names))
    print("Parameters for inference: " + ", ".join(params))
    print("")


# --- Perform some checks to make sure everything is compatible

# check that the true model provided is valid if gwobs not specified
highest_smdl_ctr=0
for channel in channels:
    base_smdls = [s.split('/')[1] for s in model_names if channel+'/' in s]
    highest_smdls = [s.split('/')[-1] for s in model_names if channel+'/' in s]
    # make sure base model is shared across channels
    if (model0.split('/')[0] not in base_smdls and not gwobs):
        raise ValueError("The true model you specified ({0:s}) is not one of the models in {1:s} directory!".format(model0, file_path))
    # make sure highest level model is given in at least one channel
    if (model0.split('/')[-1] in highest_smdls):
        highest_smdl_ctr+=1
if (highest_smdl_ctr==0 and not gwobs):
    raise ValueError("The highest level of the true model you specified ({0:s}) is not used in any of your models!".format(model0))

# ensure that the number of hyperparameters within each channel is the same depth
for channel in channels:
    channel_smdls = [x for x in model_names if channel+'/' in x]
    Nlevels_in_channel = [len(x.split('/')) for x in channel_smdls]
    if not all(x == Nlevels_in_channel[0] for x in Nlevels_in_channel):
        raise ValueError("The formation channel '{0:s}' does not have the same hierarchical levels of hyperparameters across submodels: {1:s}".format(channel, ','.join(channel_smdls)))

# ensure that models at each level are consistent across formation channels
i=1 #start at 1, which will be the highest-level hyperparameter since the formation channel is the first parameter
Nhyper_per_model = [len(x.split('/'))-1 for x in model_names]
while i <= Nhyper:
    models_at_hyperlevel = np.asarray(model_names)[np.asarray(Nhyper_per_model) >= i]
    hyper_set = sorted(set([x.split('/')[i] for x in models_at_hyperlevel]))
    for channel in channels:
        channel_smdls = [x for x in models_at_hyperlevel if channel+'/' in x]
        if len(channel_smdls) > 0:
            channel_set = sorted(set([x.split('/')[i] for x in channel_smdls]))
            if sorted(hyper_set) != sorted(channel_set):
                raise ValueError("At hyperparameter level {0:d}, the formation channel {1:s} does not have the same hyperparameters as the rest of the models (all models: {2:s}, {1:s}: {3:s}".format(i, channel, ','.join(hyper_set), ','.join(channel_set)))
    i += 1

# check that valid measurement uncertainty is specified
if gwobs:
    valid_uncertainties = ["delta", "gaussian", "posteriors"]
    if uncertainty not in valid_uncertainties:
        raise ValueError("Unspecified measurement uncertainty procedure when using GW observations: '{0:s}' (valid uncertainties: {1:s})".format(args.uncertainty, ', '.join(valid_uncertainties)))
else:
    valid_uncertainties = ["delta", "gwevents", "snr"]
    if uncertainty not in valid_uncertainties:
        raise ValueError("Unspecified measurement uncertainty procedure when using mock observations: '{0:s}' (valid uncertainties: {1:s})".format(args.uncertainty, ', '.join(valid_uncertainties)))

# If 'delta' measurement uncertainty is specified and >1 Nsamps give, spit out warning
if uncertainty=='delta' and Nsamps>1:
    warnings.warn("You specified delta-function observations but asked for more than one sample, only one sample will be used for each observations!\n")


# --- Generate observations ([observations, params, samples])
if verbose:
    print("Generating observations...\n")

# Calls gw_obs.py to generate samples if argument is passed
#model0 denotes the model data if not using true GW data
if gwobs:
    model0=None
    observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            Nsamps, uncertainty, prior)
    if verbose:
        print("Using the following {0:d} GW observations for inference:".format(len(events)))
        print(*events, sep=', ')
        print("")


# --- Freeze sample evaluations in each model. 
# This is time consuming, but only needs to be done once for each KDE model, 
# so we don't need to recompute p_model(data) over and over again when the 
# observation values aren't going to change. 
# Lower the number of samples per observation to speed this up.

if use_flows:
    for channel in channels:
        pop_models[channel].load_model(flow_model_filename, channel)
else:
    if verbose:
        print("Freezing sample evaluations in their respective models...")
    for model in model_names:
        if verbose:
            print("  {0:s}".format(model))
        getFromDict(pop_models, model.split('/')).freeze(obsdata, data_pdf=p_theta, multiproc=args.multiproc)
    if verbose:
        print("Done freezing KDE evaluations of observations!\n")

# ---  Copy kde_models so that they all have the same levels of hyperparameters
all_models_at_deepest = all([len(x.split('/')[1:])==Nhyper for x in model_names])
while all_models_at_deepest==False:
    # loop until all models have the same length
    for model in model_names:
        # See number of hyperparameters in model, subtract one for channel
        Nhyper_in_model = len(model.split('/'))-1
        if not use_flows:
            kde_hold = getFromDict(pop_models, model.split('/'))
        # loop until this model has all the hyperparam levels as well
        while Nhyper_in_model < Nhyper:
            if not use_flows:
                # remove kde model from old level
                setInDict(pop_models, model.split('/'), {})
            model_names.remove(model)
            for new_hyperparam in hyperparam_dict[Nhyper_in_model]:
                if not use_flows:
                    # copy the same kde model for the higher hyperparam level
                    new_kde = deepcopy(kde_hold)
                    new_level = model.split('/') + [new_hyperparam]
                    setInDict(pop_models, new_level, new_kde)
                # add new model name
                model_names.append(model+'/'+new_hyperparam)
            Nhyper_in_model += 1
        model_names.sort()
    # see if all models are at deepest level else repeat
    all_models_at_deepest = all([len(x.split('/')[1:])==Nhyper for x in model_names])


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


glasflow is using its own internal version of nflows


100%|██████████| 5/5 [00:10<00:00,  2.19s/it]



Formation channels: CE, CHE, GC, NSC, SMT
Astrophysical models: CE/chi00/alpha02, CE/chi00/alpha05, CE/chi00/alpha10, CE/chi00/alpha20, CE/chi00/alpha50, CE/chi01/alpha02, CE/chi01/alpha05, CE/chi01/alpha10, CE/chi01/alpha20, CE/chi01/alpha50, CE/chi02/alpha02, CE/chi02/alpha05, CE/chi02/alpha10, CE/chi02/alpha20, CE/chi02/alpha50, CE/chi05/alpha02, CE/chi05/alpha05, CE/chi05/alpha10, CE/chi05/alpha20, CE/chi05/alpha50, CHE/chi00, CHE/chi01, CHE/chi02, CHE/chi05, GC/chi00, GC/chi01, GC/chi02, GC/chi05, NSC/chi00, NSC/chi01, NSC/chi02, NSC/chi05, SMT/chi00, SMT/chi01, SMT/chi02, SMT/chi05
Parameters for inference: mchirp, q, chieff, z

Generating observations...

Using the following 46 GW observations for inference:
GW190424_180648, GW190521_074359, GW170608, GW190728_064510, GW190706_222641, GW190503_185404, GW170809, GW190602_175927, GW190519_153544, GW190719_215514, GW190413_134308, GW190910_112807, GW190727_060333, GW151226, GW170823, GW150914, GW190513_205428, GW190731_140936, GW1

In [2]:
import sys
import numpy as np
import scipy as sp
from scipy.stats import dirichlet
import pandas as pd
from functools import reduce
import operator
import pdb
from tqdm import tqdm
import time

import emcee
from emcee import EnsembleSampler

_valid_samplers = {'emcee': EnsembleSampler}

_sampler = 'emcee'
_prior = 'emcee_lnp'
_likelihood = 'emcee_lnlike'
_posterior = 'emcee_lnpost'

_nwalkers = 16
_nsteps = 10000
_fburnin = 0.2

"""
Class for initializing and running the sampler.
"""

class Sampler(object):
    """
    Sampler class.
    """
    def __init__(self, model_names, **kwargs):
        """
        model_names : list of str
            channel, chib, alpha of each eubmodel of form
            'CE/chi00/alpha02' or 'SMT/chi00'
        """

        # Store the number of population hyperparameters and formation channels
        hyperparams = list(set([x.split('/', 1)[1] for x in model_names]))
        Nhyper = np.max([len(x.split('/')) for x in hyperparams])
        channels = sorted(list(set([x.split('/')[0] for x in model_names])))

        # construct dict that relates submodels to their index number
        submodels_dict = {} #dummy index dict keys:0,1,2,3, items: particular models
        ctr=0 #associates with either chi_b or alpha (0 or 1)
        while ctr < Nhyper:
            submodels_dict[ctr] = {}
            hyper_set = sorted(list(set([x.split('/')[ctr] for x in hyperparams])))
            for idx, model in enumerate(hyper_set): #idx associates with 0,1,2,3,(4) keys
                submodels_dict[ctr][idx] = model
            ctr += 1

        # note that ndim is (Nchannels-1) + Nhyper for the model indices -- branching fractions minus 1 plus number of hyperparams
        ndim = (len(channels)-1) + Nhyper

        # store as attributes
        self.Nhyper = Nhyper
        self.model_names = model_names
        self.channels = channels
        self.ndim = ndim
        self.submodels_dict = submodels_dict


        # kwargs
        self.sampler_name = kwargs['sampler'] if 'sampler' in kwargs \
                                                            else _sampler
        if self.sampler_name not in _valid_samplers.keys():
            raise NameError("Sampler {0:s} is unknown, check valid \
samplers!".format(self.sampler_name))
        self.sampler = _valid_samplers[self.sampler_name]

        self.prior_name = kwargs['prior'] if 'prior' in kwargs else _prior
        if self.prior_name not in _valid_priors.keys():
            raise NameError("Prior function {0:s} is unknown, check valid \
priors!".format(self.prior_name))
        self.prior = _valid_priors[self.prior_name]

        self.likelihood_name = kwargs['likelihood'] if 'likelihood' in kwargs \
                                                            else _likelihood
        if self.likelihood_name not in _valid_likelihoods.keys():
            raise NameError("Likelihood function {0:s} is unknown, check \
valid likelihoods!".format(self.likelihood_name))
        self.likelihood = _valid_likelihoods[self.likelihood_name]

        self.posterior_name = kwargs['posterior'] if 'posterior' in kwargs \
                                                            else _posterior
        if self.posterior_name not in _valid_posteriors.keys():
            raise NameError("Posterior function {0:s} is unknown, check valid \
posteriors!".format(self.posterior_name))
        self.posterior = _valid_posteriors[self.posterior_name]

        self.nwalkers = kwargs['nwalkers'] if 'nwalkers' in kwargs \
                                                            else _nwalkers
        self.nsteps = kwargs['nsteps'] if 'nsteps' in kwargs else _nsteps
        self.fburnin = kwargs['fburnin'] if 'fburnin' in kwargs else _fburnin

    #still input flow dictionary
    def sample(self, pop_models, obsdata, use_flows=False, verbose=True):
        """
        Initialize and run the sampler
        """

        # --- Set up initial point for the walkers
            #ndim encompasses the population hyperparameters and the branching fractions between channels
        p0 = np.empty(shape=(self.nwalkers, self.ndim))

        # first, for the population hyperparameters
        #selects points in uniform prior for hyperparams chi_b and alpha
        for idx in np.arange(self.Nhyper):
            #TO CHANGE for continuous flows- initiate in values of chi_b and alpha range
            p0[:,idx] = np.random.uniform(0, len(self.submodels_dict[idx]), size=self.nwalkers)
        # second, for the branching fractions (we have Nchannel-1 betasin the inference because of the implicit constraint that Sum(betas) = 1
        _concentration = np.ones(len(self.channels))
        beta_p0 =  dirichlet.rvs(_concentration, p0.shape[0])
        p0[:,self.Nhyper:] = beta_p0[:,:-1]

        # --- Do the sampling
        #TO CHANGE for continuous flows - feed flows and prior range on chi_b and alpha for samplers
        posterior_args = [obsdata, pop_models, self.submodels_dict, self.channels, _concentration, use_flows] #these are arguments to pass to self.posterior
        if verbose:
            print("Sampling...")
        sampler = self.sampler(self.nwalkers, self.ndim, self.posterior, args=posterior_args) #calls emcee sampler with self.posterior as probability function
        
        for idx, result in enumerate(sampler.sample(p0, iterations=self.nsteps)): #running sampler
            print(f'{idx} iteration')
            if verbose:
                if (idx+1) % (self.nsteps/200) == 0:#progress bar
                    sys.stderr.write("\r  {0}% (N={1})".\
                                format(float(idx+1)*100. / self.nsteps, idx+1))
        if verbose:
            print("\nSampling complete!\n")

        # remove the burnin -- this removes some hyperpost samples at the start of the run before sampler equilibrates
        burnin_steps = int(self.nsteps * self.fburnin)
        self.Nsteps_final = self.nsteps - burnin_steps
        samples = sampler.chain[:,burnin_steps:,:] #chain array is number of chain, point in chain, value at that point (says in model_select?)
        lnprb = sampler.lnprobability[:,burnin_steps:]

        # synthesize last betas, since they sum to unity
        last_betas = (1.0-np.sum(samples[...,self.Nhyper:], axis=2))
        last_betas = np.expand_dims(last_betas, axis=2)
        samples = np.concatenate((samples, last_betas), axis=2)

        self.samples = samples
        self.lnprb = lnprb



# --- Define the likelihood and prior

def lnp(x, submodels_dict, _concentration):
    """
    Log of the prior. 
    Returns logL of -inf for points outside, uniform within. 
    Is conditional on the sum of the betas being one.
    """
    # first get prior on the hyperparameters, flat between the model (dummy) indices
    for hyper_idx in list(submodels_dict.keys()):
        hyperparam = x[hyper_idx]
        if ((hyperparam < 0) | (hyperparam > len(submodels_dict[hyper_idx]))):
            return -np.inf

    # second, get the prior on the betas as a Dirichlet prior
    betas_tmp = np.asarray(x[len(submodels_dict):])
    betas_tmp = np.append(betas_tmp, 1-np.sum(betas_tmp)) #synthesize last beta
    if np.any(betas_tmp < 0.0):
        return -np.inf
    if np.sum(betas_tmp) != 1.0:
        return -np.inf

    # Dirchlet distribution prior for betas
    return dirichlet.logpdf(betas_tmp, _concentration)


def lnlike(x, data, pop_models, submodels_dict, channels, use_flows): #data here is obsdata previously, and x is the point in log hyperparam space
    """
    Log of the likelihood. 
    Selects on model, then tests beta.
    """
    start = time.time()
    model_list = []
    hyperparam_idxs = []
    for hyper_idx in list(submodels_dict.keys()):
        hyperparam_idxs.append(int(np.floor(x[hyper_idx])))
        model_list.append(submodels_dict[hyper_idx][int(np.floor(x[hyper_idx]))]) #finds where walker is in hyperparam space

    #print(f'calculated position {time.time()-start}')    
    # get detectable betas
    betas_tmp = np.asarray(x[len(submodels_dict):])
    betas_tmp = np.append(betas_tmp, 1-np.sum(betas_tmp))

    # Likelihood
    prob = np.zeros(data.shape[0])

    # Detection effiency for this hypermodel
    alpha = 0

    # Iterate over channels in this submodel, return cached values of likelihood in 4d KDE
    for channel, beta in zip(channels, betas_tmp):
        model_list_tmp = model_list.copy()
        model_list_tmp.insert(0,channel) #list with channel, 2 hypermodels (chi_b, alpha)
        if use_flows:
            smdl = pop_models[channel]
            #print(f'grabbed channel {time.time()-start}')    
            prob += beta * smdl(data, hyperparam_idxs)
            #print(f'calced prob {time.time()-start}')    
        else:
            smdl = reduce(operator.getitem, model_list_tmp, pop_models) #grabs correct submodel
            prob += beta * smdl(data)
        #calls popModels __call__(data) to return likelihood.
        # add contribution from this channel
        alpha += beta * smdl.alpha

    return np.log(prob/alpha).sum()


def lnpost(x, data, kde_models, submodels_dict, channels, _concentration, use_flows):
    """
    Combines the prior and likelihood to give a log posterior probability 
    at a given point

    x : ?
        walker points in hyperparameters space to sample probability
    data : ?
        GW observations
    kde_models : Dict
        models of KDE probabilities
    """
    # Prior
    log_prior = lnp(x, submodels_dict, _concentration)
    if not np.isfinite(log_prior):
        return log_prior

    # Likelihood
    log_like = lnlike(x, data, kde_models, submodels_dict, channels, use_flows)


    return log_like + log_prior #evidence is divided out




_valid_priors = {'emcee_lnp': lnp}
_valid_likelihoods = {'emcee_lnlike': lnlike}
_valid_posteriors = {'emcee_lnpost': lnpost}


In [3]:
sampler = Sampler(model_names)
sampler.sample(pop_models, obsdata, use_flows, verbose=verbose)

Sampling...
start 0.0004982948303222656
calc logprob 0.3275110721588135
numpy logprob 0.32869911193847656
start 2.47955322265625e-05
calc logprob 0.01577591896057129
numpy logprob 0.016380786895751953
start 1.621246337890625e-05
calc logprob 0.017368078231811523
numpy logprob 0.01801013946533203
start 1.6927719116210938e-05
calc logprob 0.014883756637573242
numpy logprob 0.015594959259033203
start 2.5033950805664062e-05
calc logprob 0.014715909957885742
numpy logprob 0.015398740768432617
start 1.621246337890625e-05
calc logprob 0.019274234771728516
numpy logprob 0.019939184188842773
start 1.6927719116210938e-05
calc logprob 0.016717195510864258
numpy logprob 0.017340898513793945
start 1.9073486328125e-05
calc logprob 0.01782989501953125
numpy logprob 0.018100976943969727
start 1.3828277587890625e-05
calc logprob 0.017372846603393555
numpy logprob 0.018030881881713867
start 1.6689300537109375e-05
calc logprob 0.013765811920166016
numpy logprob 0.014393806457519531
start 1.40666961669921

Traceback (most recent call last):
  File "/Users/stormcolloms/opt/anaconda3/envs/amaze/lib/python3.9/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/var/folders/t1/pzgx3lnj70l28d88_yvswpsr0000gq/T/ipykernel_19842/597299461.py", line 238, in lnpost
    log_like = lnlike(x, data, kde_models, submodels_dict, channels, use_flows)
  File "/var/folders/t1/pzgx3lnj70l28d88_yvswpsr0000gq/T/ipykernel_19842/597299461.py", line 208, in lnlike
    prob += beta * smdl(data, hyperparam_idxs)
  File "/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/populations/Flowsclass_dev.py", line 359, in __call__
    mapped_obs = self.map_obs(obs)
  File "/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/populations/Flowsclass_dev.py", line 387, in map_obs
    mapped_data[i,3],_,_= self.logistic(sample[3], True, False, self.mappings[4], self.mappings[5])
KeyboardInterrupt


KeyboardInterrupt: 